In [193]:
import pandas as pd
import math
import copy
import numpy as np 
import itertools
import more_itertools as mit
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
import string
import re
from IPython import get_ipython

get_ipython().magic('run -i "2_basic_retrieval.ipynb"')
get_ipython().magic('run -i "1_preprocessing_corpus_queries.ipynb"')

## Tiered index structure

In [174]:
# Tiered index
def tiered_index(corpus, chunks):
    
    #print('Function is tested on term \'ddt\'. It performs following steps:')
    
    tf_dict = tf(corpus)
    
    def tf_inverted_index(tf_dict):
        tf_ii_dict = {}
        for doc in tf_dict:
            for term in tf_dict[doc]:
                if term not in tf_ii_dict:
                    inner_dict = {}
                    tf_ii_dict[term] = inner_dict
                    inner_dict[doc] = tf_dict[doc][term]
                else:
                    tf_ii_dict[term][doc] = tf_dict[doc][term]
        return tf_ii_dict
    
    tf_ii_dict = tf_inverted_index(tf_dict)
    #print("\nInverted index:")
    #print(tf_ii_dict["ddt"])
    
    def sort_dict(tf_ii_dict):
        for doc in tf_ii_dict:
             tf_ii_dict[doc] = {k: v for k, v in sorted(tf_ii_dict[doc].items(), 
                                                        key=lambda item: item[1], reverse=True)}
        return tf_ii_dict
    
    
    tf_ii_dict_sorted = sort_dict(tf_ii_dict)
    #print("\nSorted inverted index by tf(term, doc):")
    #print(tf_ii_dict_sorted["ddt"])
    
    def transform_dict(tf_ii_dict_sorted):
        new = {}
        for k,v in tf_ii_dict_sorted.items():
            new[k] = list(v)
        return new
    
    transformed = transform_dict(tf_ii_dict_sorted)
    #print("\nSorted inverted index without tf(term,doc) values:")
    #print(transformed["ddt"])
    
    def chunk_list(lst, chunks):
        return [list(x) for x in mit.divide(chunks, lst)]
    
    def chunk_dict(transformed, chunks):
        for term in transformed:
            doc_chunks = chunk_list(transformed[term],chunks)
            new = {}
            for i in range(0,len(doc_chunks)):
                new[i] = doc_chunks[i]
            transformed[term] = new
        return transformed
    
    tf_ii_dict_sorted = chunk_dict(transformed, chunks)
        
    #print("\nChunked inverted index:")
    #print(tf_ii_dict_sorted["ddt"])
    
    def sort_chunks(tf_ii_dict_sorted):
        for term, tier in tf_ii_dict_sorted.items():
            for tier, lst in tf_ii_dict_sorted[term].items():
                lst.sort()
        return tf_ii_dict_sorted
    
    tf_ii_dict_sorted = sort_chunks(tf_ii_dict_sorted)
    
    #print("\nChunked inverted index with sorted chunks (tiered index):")
    #print(tf_ii_dict_sorted["ddt"])
    return tf_ii_dict_sorted

## TEST Tiered index structure

In [175]:
#Call the function on the corpus
tiered_index_dict = tiered_index(corpus, 4) 

## Intersection algorithm

In [111]:
def inter_one_list(p1,p2): #posting 1 list, posting 2 list
    i=0
    j=0
    intersection = []
    
    while i < len(p1) and j < len(p2):
        if p1[i] == p2[j]:
            if i== 0 or p1[i] != p1[i-1]:
                intersection.append(p1[i])
            i += 1
            j += 1           
        elif p1[i] < p2[j]:
            i += 1
        else: # p[i] > p[j]
            j += 1     
    return intersection

# first sort list in ascending order
# take first two elements and produce output
# delete these two merged lists from super list
# append result of merge
def inter_n_lists(lst):
    
    rank_lst = sorted(lst, key = len)   
    
    if len(rank_lst) == 0:
        rank_lst = rank_lst

    if len(rank_lst) <= 1 and len(rank_lst) > 0:
        intersection = rank_lst[0]
    
    while len(rank_lst) > 1:
        intersection = inter_one_list(rank_lst[0], rank_lst[1])
        del rank_lst[:2]
        rank_lst.append(intersection)
        rank_lst = sorted(rank_lst, key = len)
                
    return rank_lst

## Tiered index retrieval

In [225]:
def tired_index_retrieve(tiered_index_dict, queries, query_index, doc_vectors, idf_dict, tieres_no, top_k):
    
    query = queries['TEXT'][query_index]   
    
    def retrieve_postings(query): 
        postings = []
        for i in range(0,len(query)): 
            try:
                dic = {}
                dic[query[i]] = tiered_index_dict[query[i]]
                postings.append(dic)           
            except KeyError:
                pass
        return postings

    def get_tieres(postings, t):
        tieres_lst = []
        for i in range(len(postings)): 
            d = postings[i]
            key = [key for key in d.keys()][0]  
            element = d[key][t]
            tieres_lst.append(element)
        return tieres_lst
    
    if len(query) == 0:
        print("\nQuery is empty!")
    else:
        print("\nQuery: ", query)
        postings = retrieve_postings(query)
    
        t=-1
        tieres = [[]] * len(postings)
        common_docs = []
    
        while (len(common_docs) < top_k) and t+1 < tieres_no:
        
            next_tieres = get_tieres(postings, (t+1)) # get next tieres        
        
            for i in range(len(tieres)): # merge them with previous tieres
                tieres[i] = tieres[i] + next_tieres[i] 
                tieres[i].sort() # we need to sort merged tieres!!!
        
            common_docs = inter_n_lists(tieres) # intersection

            t+=1   
     
        # Calculate cosine similariy between query vector and outputed documents (common_docs)
    
        # if no common (or not enough) documents were found perform basic retrieval
    
        q_vector = build_q_vector(query, doc_vectors, idf_dict)
    
        if len(common_docs[0]) < top_k:
            print("\nNo documents find via tired index, basic retrieval performed.")
            return basic_retrieve(q_vector = q_vector,
                      doc_vectors = doc_vectors, 
                      top_k = 5,
                      idf_dict = idf_dict)
    
        else:
            print("\nDocuments found via tired index")
            doc_vectors_cropped = doc_vectors[doc_vectors.index.isin(common_docs[0])]
        
            return basic_retrieve(q_vector = q_vector,
                                  doc_vectors = doc_vectors_cropped, 
                                  top_k = top_k,
                                  idf_dict = idf_dict)

## TEST Tiered index retrieval (3 tieres, find top 3 documents for first 3 queries)

In [216]:
# load corpus 
corpus = pd.read_csv('nfcorpus/dev.docs', sep='\t', names=['ID', 'TEXT'])

# load queries (titles)
queries = preprocess_queries(corpus, pd.read_csv('nfcorpus/dev.titles.queries', sep='\t', names=['ID', 'TEXT']))

In [ ]:
# get needed arguments
doc_vectors = build_doc_vectors(corpus)
idf_dict = idf(corpus)
no_tieres = 3
tiered_index_dict = tiered_index(corpus,4)


# retrieve
for i in range(10):
    test = tired_index_retrieve(tiered_index_dict = tiered_index_dict , 
                            queries = queries, 
                            query_index = i, 
                            doc_vectors = doc_vectors, 
                            idf_dict = idf_dict, 
                            tieres_no = no_tieres, 
                            top_k = 3)
    print(test)


Query:  ['deep', 'food', 'may', 'cancer']

No documents find via tired index, basic retrieval performed.
            ID                                               TEXT
1302  MED-2697  impaired endothelial function meal rich cookin...
1965  MED-3703  association allergies cancer pubmed ncbi abstr...
582   MED-1721  cancer incidence mortality relation body mass ...
132   MED-1151  organic food consumption incidence cancer larg...
1961  MED-3699  concordance world cancer research fund/america...

Query:  ['ddt']

Documents found via tired index
        ID                                               TEXT
0  MED-118  alkylphenols human milk relations dietary habi...
2  MED-330  dietary phosphorus acutely impairs endothelial...
5  MED-335  differences total vitro digestible phosphorus ...

Query:  ['treat', 'diet']
